In [2]:
class SlidingPuzze():
  def __init__(self, num_blocks):
    self.num_blocks = num_blocks # tamanho do puzze que é 3/3 = 9 peças

  #procurando um numero na matriz por exemplo aonde esta o numero 5 na Matriz que pode ser na linha 2 coluna 1 [2,1] =5
  def _search_position(self, state , element):
    for i in range(self.num_blocks):  
      for j in range(self.num_blocks):
        if state[i,j] == element: # exemplo se [2,3] == 5 
          return i,j
    return None, None  #nao encontro o elemento na matriz

  #verificando se cheguei no que quero no jogo / estado objetivo onde todas peças estao corretas 
  def verify_states(self, current , target):  #current  = estado atual  , target = estado objetivo
    flag = True # esta no estado objetivo?

    #for das linhas
    for i in range(self.num_blocks):
      if not flag: # se a flag ja tiver com false ja pode parar esse for
        break

      for j in range(self.num_blocks):
        # se a matriz na posição i j tiver diferente do objetivo ja pode parar que nao é o objetivo
        if current[i,j] != target[i,j]:
          flag = False 
          break # ja sai do segundo for e volta pro primeiro

    return flag


  # pra expandir o estado ver a possibilidade de movimento da matriz
  def expand_state(self, current):
    new_states = []

    line, column = self._search_position(current, 0) # procurando aonde esta o ZERO na matriz e guardando na linha,coluna

    #provavelmente entrara em todos IFS
    #cima
    if line >0: # pra nao pegar posição -1 q nao existe na matriz
      new_line = line -1 #nova linha onde vai o ZERO
      new_state = np.copy(current)  # copiando a matriz atual

      target_block = new_state[new_line, column] # armazenando o valor que esta nessa posiçao |||exemplo target_block=new_state[0,1] = 5
      new_state[new_line, column] = 0            # colocando o zero na nova posiçao, que tirou o numero |||exemplo new_state[0,1] = 0
      new_state[line, column] = target_block     # colocando o valor aonde estava o zero |||exemplo new_state[1,1] = 5

      new_states.append(new_state) # novo estado da matriz


     #baixo
    if line<self.num_blocks-1:  
      new_line = line + 1 
      new_state = np.copy(current)  

      target_block = new_state[new_line, column] 
      new_state[new_line, column] = 0            
      new_state[line, column] = target_block    

      new_states.append(new_state) 

     #esquerda
    if column>0:  
      new_column = column - 1 
      new_state = np.copy(current)  

      target_block = new_state[line, new_column] 
      new_state[line, new_column] = 0            
      new_state[line, column] = target_block    

      new_states.append(new_state) 

    #Diretita
    if column>self.num_blocks-1:  
      new_column = column + 1 
      new_state = np.copy(current)  

      target_block = new_state[line, new_column] 
      new_state[line, new_column] = 0            
      new_state[line, column] = target_block    

      new_states.append(new_state) 
    return new_states

  #IMPLEMENTAR DISTANCIA DE PEÇAS NAS POSIÇÕES ERRADAS() a quantidade de peças erradas em relação a um estado objetivo
  def manhattanDistance(self, current, target ): # somas dos movimentos dos elemtos ate o lugar certo 
    total_dist = 0

    for i in range(self.num_blocks):
      for j in range(self.num_blocks):
        element = current [i , j] #pegando o elemto atual na matriz
        line, column = self._search_position(target , element) # vendo a posição do elemento na matriz

        dist =  abs(line - i) + abs(column - j)  # abs() pra nao dar negativo, dist do elemento ate sua posição correta
        total_dist +=dist

    return total_dist # retornando o total de distacia dos elemtos ao objetivo delas (posição correta)

In [3]:
import heapq

class GreedySearch():
  def __init__(self, problem):
    self.problem = problem

  def _verify_visited(self, state , states_visited):  #vendo se o estado foi visitado
    for i in states_visited:
      if self.problem.verify_states(i, state):
        return True
    return False

  def search(self, start, target):
    p_queue = [] # lista de prioridades
                            #(valor da euristica, id , puzzle inical)
    heapq.heappush(p_queue , (0 , 0 , start))

    state_id = 0
    solution_found = False
    states_visited = []
    count_states = 0

    while len(p_queue) != 0:
      current  = heapq.heappop(p_queue)[2]
      states_visited.append(current)   # colocando o estado atual nos visitados para nao acesar futuramente de novo

      if self.problem.verify_states(current, target):
        solution_found = True
        break
      else:
        count_states +=1
        print(f"visitando #{count_states}")

        new_state =  self.problem.expand_state(current) # espandindo pros novos estados 

        for i in new_state: # verificando os estatos acima
          if not self._verify_visited(i , states_visited):  # verifico se ja foi visitado | i==estado novo
            state_id += 1
            priority = self.problem.manhattanDistance(i, target ) # vendo a distancia de manhantan de 1 = estado novo , e enviando o estado objetivo=target

            print(f"{i}\nh= {priority}")

            heapq.heappush(p_queue, (priority , state_id, i))

    return solution_found, states_visited, count_states

In [4]:
import numpy as np

problem = SlidingPuzze(3)

start = np.matrix([[2, 8, 3], [1, 6, 4], [7, 5, 0]])
target = np.matrix([[1, 2, 3], [8, 0, 4], [7, 6, 5]])



In [ ]:
Greedy_search = GreedySearch(problem)
solution, states_visited , steps = Greedy_search.search(start, target)

if solution:
  print(f"Solucao encontrada com {steps} passos " )
else:
  print("Solucao nao encontrada")